In [17]:
import requests
import json
from qsforex import settings
from datetime import datetime, timedelta
import csv
'''
This is to learn how to query OANDA for historical price patterns. 
see here for more details:
<http://developer.oanda.com/rest-live/rates/#retrieveInstrumentHistory>

The following example illustrates the request format with curl:
"https://api-fxtrade.oanda.com/v1/candles?instrument=EUR_USD&count=2&candleFormat=midpoint&granularity=D&dailyAlignment=0&alignmentTimezone=America%2FNew_York"
Here's a breakdown:
domain: api-fxtrade.oanda.com
endpoint: /v1/candles
instrument: (req.)pairs requested
count: # of candles (opt., default 500)
candleFormat: (opt.,) bidask (default) returns both bid and ask candles
              midpoint returns midpoint between bid and ask
granularity: (opt., default=S5) time range represented by ea. candlestick
dailyAlignment: (opt.,)The hour of day used to align candles with hourly, 
                daily, weekly, or monthly granularity, defaults to 1700, NY time.
alignmentTimezone: 
                (opt.,default is America/NewYork)
                The timezone to be used for the dailyAlignment parameter.
                default is America/NewYork
                This parameter does NOT affect the returned timestamp: 
                        START or END PARAMETERS WILL ALWAYS BE IN UTC.
                Full list available here: 
                    http://developer.oanda.com/docs/timezones.txt
start(opt.): start time stamp
end (opt.)): end time stamp
weeklyAlignment (opt.,default; "Friday":
                The day of the week used to align candles with weekly granularity. 
                The value specified will be used as the start/end day when calculating 
                the weekly candles. Valid values are: “Monday”, “Tuesday”, “Wednesday”, 
                “Thursday”, “Friday”, “Saturday”, “Sunday”.
              

'''


# Some preliminary setup:
targetDomain = 'practice'
domain = settings.ENVIRONMENTS['api'][targetDomain]
access_token = settings.ACCESS_TOKEN
account_id = settings.ACCOUNT_ID
INST = 'EUR_USD'
endpoint = '/v1/candles'
DAYS = 5
GRAN = 'M15'


In [ ]:
    
def GetGranularitySecs(granularity):
    '''
    Parses granularity, specified like S10 or M15 into corresponding 
    number of seconds
    '''
    if granularity[0] == 'S':
        return int(granularity[1:])
    elif granularity[0] == 'M':
        # minute
        if len(granularity) > 1:
            return 60*int(granularity[1:])
        else:
        # month: does not take into account actual month length
            return 60*60*24*30
    elif granularity[0] == 'H':
        return 60*60*int(granularity[1:])
    elif granularity[0] == 'D':
        return 60*60*24
    elif granularity[0] == 'W':
        return 60*60*24*7

In [23]:
def PrepReq(pair,granularity='S5', candle_format="bidask",
            count=None, daily_alignment=None, alignment_timezone=None,
            weekly_alignment="Monday", start=None, end=None):
    params = {
            "accountId": account_id,
            "instrument": pair,
            "candleFormat": candle_format,
            "granularity": granularity,
            "count": count,
            "dailyAlignment": daily_alignment,
            "alignmentTimezone": alignment_timezone,
            "weeklyAlignment": weekly_alignment,
            "start": start,
            "end": end,
        }
    client = requests.Session()
    requests.packages.urllib3.disable_warnings()
    url = "https://" + domain + "/v1/candles"
    headers = {'Authorization': 'Bearer ' + access_token}
    req = requests.Request('GET', url,headers=headers,params=params)
    pre = req.prepare()
    resp = client.send(pre, stream=False, verify=True)
    if resp.ok:
        rjson = resp.json()
        return rjson
    else:
        print(resp.text)

In [24]:
start = datetime.now() - timedelta(days=DAYS)
end = start + timedelta(hours=1)
start=start.strftime("%Y-%m-%dT%H:%M:%S.%f%z")
end=end.strftime("%Y-%m-%dT%H:%M:%S.%f%z")
msg = PrepReq(INST,start=start,end=end) 

In [26]:
import pandas as pd

In [55]:
df = pd.DataFrame(msg['candles'])
df.index = pd.to_datetime(df['time'])
df.head()

,closeAsk,closeBid,complete,highAsk,highBid,lowAsk,lowBid,openAsk,openBid,time,volume
time,,,,,,,,,,,
2016-09-13 18:49:05,1.12100,1.12087,True,1.12100,1.12087,1.12098,1.12084,1.12098,1.12084,2016-09-13T18:49:05.000000Z,2
2016-09-13 18:49:30,1.12103,1.12089,True,1.12103,1.12089,1.12103,1.12089,1.12103,1.12089,2016-09-13T18:49:30.000000Z,1
2016-09-13 18:49:35,1.12105,1.12091,True,1.12105,1.12091,1.12105,1.12091,1.12105,1.12091,2016-09-13T18:49:35.000000Z,1
2016-09-13 18:49:40,1.12110,1.12096,True,1.12110,1.12096,1.12108,1.12093,1.12108,1.12093,2016-09-13T18:49:40.000000Z,2
2016-09-13 18:49:45,1.12106,1.12093,True,1.12106,1.12093,1.12106,1.12093,1.12106,1.12093,2016-09-13T18:49:45.000000Z,1
